In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sktime.transformations.panel.rocket import Rocket
from datetime import datetime, timedelta
from sklearn.linear_model import RidgeCV
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # only for pandas version < 1.9
from sklearn.model_selection import train_test_split
from scipy.fft import fft


## Create Dataframe with Label

In [2]:
features=['x',"y","z"]
df_features = pd.read_parquet("df_pump.parquet")


In [3]:
df_features

,x,y,z,time,label
0,2035,-74,58,2023-01-10 10:03:00,3610.131348
1,2044,-65,61,2023-01-10 10:03:00,3610.131348
2,2048,-73,71,2023-01-10 10:03:00,3610.131348
3,2052,-72,81,2023-01-10 10:03:00,3610.131348
4,2056,-72,76,2023-01-10 10:03:00,3610.131348
...,...,...,...,...,...
3371515,2084,-58,77,2023-01-10 13:15:10,3699.336914
3371516,2077,-70,93,2023-01-10 13:15:10,3699.336914
3371517,2084,-70,106,2023-01-10 13:15:10,3699.336914
3371518,2072,-58,112,2023-01-10 13:15:10,3699.336914


## Preprocess data for prediction

In [4]:
# put data into a dataframe as expected by ROCKET
def transform_to_rocket_1feature(df:pd.DataFrame,id):
    df_Rocket=pd.DataFrame(columns=[id])
    df_Y=pd.DataFrame(columns=[id])
    df_Time=pd.DataFrame(columns=['time'])
   
    for g in df.groupby(by=['time']):
        indxC=df_Rocket.index.max()
        if indxC!=indxC:
            indxC=0
        else:
            indxC=indxC+1
        df_Rocket.loc[indxC]= [g[1][id]]
        df_Y=df_Y.append({id: g[1]['label'].iloc[0]}, ignore_index=True)
        df_Time=df_Time.append({'time': g[1]['time'].iloc[0]}, ignore_index=True)
    return df_Rocket,df_Y,df_Time

def transform_to_rocket(df,features):
    dfX=None
    dfY=None
    dfTime=None
    for id in features:
        dfX_,dfY_,dfTime_=transform_to_rocket_1feature(df,id)
        if dfX is None:
            dfX=dfX_
            dfY=dfY_
            dfTime=dfTime_
        else:
            dfX[id]=dfX_[id]
            
    return dfX, dfY, dfTime

In [5]:
# transform training data to ROCKET df
X_train,dfY_train,dfTime_train=transform_to_rocket(df_features,features)

X_train, X_test, y_train, y_test, time_train, time_test = train_test_split(X_train, dfY_train, dfTime_train, test_size=0.35, random_state=42, shuffle=False)

In [6]:
def convert_to_fft(df, N=512):
    """
    FFT transformation
    """
    pd_list = []
    for row in df.iterrows():
        var_splits = []
        for array in row[1]:
            yf = fft(array)

            var_splits.append(np.abs(yf)[1:int(N/2)])
            
        pd_list.append(np.concatenate(var_splits))
        
    transform = pd.DataFrame(pd_list)
    return transform

In [7]:
X_train_transform = convert_to_fft(X_train)
X_test_transform = convert_to_fft(X_test)

## Train the model

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score

In [9]:
# Fit ridgecv regression model
rm=RidgeCV(alphas=np.logspace(-3, 3, 10))
rm.fit(X_train_transform, y_train)

score_train=rm.score(X_train_transform, y_train)
predictions_train=rm.predict(X_train_transform)

In [10]:
# Fit random forest regression model
regr = RandomForestRegressor() 
regr.fit(X_train_transform, y_train.to_numpy().ravel())

score_train_regr=regr.score(X_train_transform, y_train.to_numpy().ravel())
predictions_train_regr=regr.predict(X_train_transform)

In [11]:
score_test_regr=regr.score(X_test_transform, y_test)
predictions_test_regr=regr.predict(X_test_transform)

In [12]:
score_test=rm.score(X_test_transform, y_test)
predictions_test=rm.predict(X_test_transform)

## Messure modell performance

In [13]:
from sklearn.metrics import mean_squared_error, r2_score
print("Linear regression:")
print("RMSE for trainset:", mean_squared_error(y_train, predictions_train, squared=False))
print("RMSE for testset:", mean_squared_error(y_test, predictions_test, squared=False))
print(" ")
# Random forest regressor
print("Random forest regressor:")
print("RMSE for trainset:", mean_squared_error(y_train, predictions_train_regr, squared=False))
print("RMSE for testset:", mean_squared_error(y_test, predictions_test_regr, squared=False))

Linear regression:
RMSE for trainset: 821.8461910033084
RMSE for testset: 1209.2489158530518
 
Random forest regressor:
RMSE for trainset: 191.54991776923228
RMSE for testset: 888.0572077567085


In [14]:
# Linear regression
print("Linear regression:")
print('Score for training data, features {}: {}'.format(features, score_train))
print('Score for test data, features {}: {}'.format(features, score_test))
print(" ")
# Random forest regressor
print("Random forest regressor:")
print('Score for training data, features {}: {}'.format(features, score_train_regr))
print('Score for test data, features {}: {}'.format(features, score_test_regr))

Linear regression:
Score for training data, features ['x', 'y', 'z']: 0.9702281461887281
Score for test data, features ['x', 'y', 'z']: 0.8664591479874488
 
Random forest regressor:
Score for training data, features ['x', 'y', 'z']: 0.9983827069352644
Score for test data, features ['x', 'y', 'z']: 0.9279780930139978


## Plotting model results

In [15]:
df_pred = pd.DataFrame(predictions_train, columns=['Prediction'])
label = y_train.reset_index()
label.drop(columns="index", inplace=True)
label.rename(columns={"x":"Label"}, inplace=True)
time_train.sort_values(by="time", inplace=True)
df_train = pd.merge(df_pred, label, left_index=True, right_index=True)

In [16]:
fig = px.line(df_train, y=["Prediction", "Label"], x=time_train["time"], title="Regression model prediction vs true value on train data",
labels={"variable": "Variable"})
fig.update_layout(
    xaxis_title="Time",
    yaxis_title = "Value"

    )


fig.show()

In [17]:
df_predt = pd.DataFrame(predictions_test, columns=['Prediction'])
labelt = y_test.reset_index()
labelt.drop(columns="index", inplace=True)
labelt.rename(columns={"x":"Label"}, inplace=True)
time_test.sort_values(by="time", inplace=True)
df_test = pd.merge(df_predt, labelt, left_index=True, right_index=True)

In [18]:
fig = px.line(df_test, y=["Prediction", "Label"], x=time_test["time"], title="Regression model prediction vs true value on test data",
             labels={"variable": "Variable"})
fig.update_layout(
    xaxis_title="Time",
    yaxis_title = "Value"

    )


fig.show()

In [19]:
df_predrf = pd.DataFrame(predictions_train_regr, columns=['Prediction'])
label = y_train.reset_index()
label.drop(columns="index", inplace=True)
label.rename(columns={"x":"Label"}, inplace=True)
time_train.sort_values(by="time", inplace=True)
df_trainrf = pd.merge(df_predrf, label, left_index=True, right_index=True)

In [20]:
df_predrft = pd.DataFrame(predictions_test_regr, columns=['Prediction'])
label = y_test.reset_index()
label.drop(columns="index", inplace=True)
label.rename(columns={"x":"Label"}, inplace=True)
time_train.sort_values(by="time", inplace=True)
df_testrf = pd.merge(df_predrft, label, left_index=True, right_index=True)

In [21]:
fig = px.line(df_trainrf, y=["Prediction", "Label"], x=time_train["time"], title="Random forest model prediction vs true value on train data",
             labels={"variable": "Variable"})
fig.update_layout(
    xaxis_title="Time",
    yaxis_title = "Value"

    )


fig.show()

In [22]:
fig = px.line(df_testrf, y=["Prediction", "Label"], x=time_test["time"], title="Random forest model prediction vs true value on test data",
             labels={"variable": "Variable"})
fig.update_layout(
    xaxis_title="Time",
    yaxis_title = "Value"

    )


fig.show()

## Deep neuroal network

In [23]:
from torch import nn
import torch
from sklearn.preprocessing import StandardScaler

In [24]:
class NN(nn.Module):
  '''
    Deep neural Network for Regression
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(765, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 1),
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [26]:
train_x = torch.Tensor(X_train_transform.to_numpy()).float().to(device)
test_x = torch.Tensor(X_test_transform.to_numpy()).float().to(device)
train_y = torch.Tensor(y_train.to_numpy()).float().to(device)
test_y = torch.Tensor(y_test.to_numpy()).float().to(device)

In [27]:
import time

dnn = NN()
dnn.to(device)    
dnn.train()      


learning_rate = 0.0000000001

loss_func = nn.MSELoss()  # no softmax activation in the network, it needs raw data for the loss 

optimizer = torch.optim.SGD(dnn.parameters(), lr=learning_rate, momentum = 0.9) # execute GD, gets (parameters aka weights, and learning rate, with momentum)

start_time = time.time()
losses = []
test_losses = []

In [28]:
no_epochs = 4000

for iteration in range(no_epochs):
    
    optimizer.zero_grad() # set gradients of all tensor to 0 
    y_hat = dnn(train_x) # we predict on all data points (= batch gradient descent) 
    
    loss = loss_func(y_hat, train_y) # calculate the loss
    loss.backward() # backpropagate the loss to calculate gradients
    optimizer.step() # update the weights using these gradients 
    
    losses.append(loss.item())
    
    if iteration % 20 == 0:
        test_loss = loss_func(dnn(test_x), test_y)
        test_losses.append(test_loss.item())
        print(f"Loss in epoch {iteration} is {loss.item()} and test loss is {test_loss.item()}")

Loss in epoch 0 is 200416256.0 and test loss is 96739456.0
Loss in epoch 20 is 28170050.0 and test loss is 26150718.0
Loss in epoch 40 is 11160935.0 and test loss is 20184554.0
Loss in epoch 60 is 8522912.0 and test loss is 19653854.0
Loss in epoch 80 is 7326589.0 and test loss is 16271597.0
Loss in epoch 100 is 6548839.0 and test loss is 14236728.0
Loss in epoch 120 is 5900052.0 and test loss is 12682313.0
Loss in epoch 140 is 5332331.5 and test loss is 11375609.0
Loss in epoch 160 is 4826072.5 and test loss is 10310190.0
Loss in epoch 180 is 4389676.5 and test loss is 9409422.0
Loss in epoch 200 is 4016440.0 and test loss is 8658449.0
Loss in epoch 220 is 3691943.75 and test loss is 8083835.5
Loss in epoch 240 is 3403178.0 and test loss is 7652130.5
Loss in epoch 260 is 3147899.5 and test loss is 7265006.0
Loss in epoch 280 is 2922279.75 and test loss is 6912787.0
Loss in epoch 300 is 2722294.25 and test loss is 6618442.0
Loss in epoch 320 is 2547875.75 and test loss is 6340028.5
Los

In [39]:
NN_r2tr = r2_score(dnn(train_x).detach().numpy(), 
         train_y.numpy()
         )
NN_r2te = r2_score(dnn(test_x).detach().numpy(), 
         test_y.numpy()
         )
print("Neural network:")
print('Score for training data, features {}: {}'.format(features, NN_r2tr))
print('Score for training data, features {}: {}'.format(features, NN_r2te))
print(" ")
print("RMSE for trainset:", mean_squared_error(dnn(train_x).detach().numpy(), train_y.numpy(), squared=False))
print("RMSE for testset:", mean_squared_error(dnn(test_x).detach().numpy(), test_y.numpy(), squared=False))

Neural network:
Score for training data, features ['x', 'y', 'z']: 0.9865150326596721
Score for training data, features ['x', 'y', 'z']: 0.8515615663539386
 
RMSE for trainset: 556.1459
RMSE for testset: 1733.2671


## Plotting the modell results

In [30]:
fig = px.line(x=range(0, no_epochs), y=losses, title="Network loss over number of epochs")
fig.update_layout(
    xaxis_title="Number of epchs",
    yaxis_title = "Loss"
)
fig.show()

In [31]:
fig = px.line(x=range(0, 200), y=test_losses, title="Network loss over number of epochs unseen data")
fig.update_layout(
    xaxis_title="Number of epchs",
    yaxis_title = "Loss"
)
fig.show()

In [32]:
dnn.eval()
y_train_pred = dnn(train_x)


In [33]:
y_train_pred_list = [line.item() for line in y_train_pred]
train_y_list = [line.item() for line in train_y]

In [34]:
fig = px.line(x=time_train["time"], y=[y_train_pred_list, train_y_list], title="Model prediction vs true value on test data",
              labels={"variable": "Variable"})
newnames = {'wide_variable_0':'Prediction', 'wide_variable_1': 'Label'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.update_layout(
    xaxis_title="Time",
    yaxis_title = "Value"

    )

fig.show()


In [35]:
dnn.eval()
y_test_pred = dnn(test_x)

In [36]:
y_test_pred_list = [line.item() for line in y_test_pred]
test_y_list = [line.item() for line in test_y]

In [37]:
fig = px.line(x=time_test["time"], y=[y_test_pred_list, test_y_list], title="Model prediction vs true value on test data",
              labels={"variable": "Variable"})
newnames = {'wide_variable_0':'Prediction', 'wide_variable_1': 'Label'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.update_layout(
    xaxis_title="Time",
    yaxis_title = "Value"

    )

fig.show()
